In [ ]:
import warnings
warnings.filterwarnings('ignore')

import time
import os
import re
import pandas as pd
import random

import matplotlib as mpl
from matplotlib import rc, rcParams

import numpy as np
from numpy import ndarray

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from scipy.stats import truncnorm

import multiprocessing
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (Input, Dropout, LSTM, Reshape, LeakyReLU,
                          Concatenate, ReLU, Flatten, Dense, Embedding,
                          BatchNormalization, Activation, SpatialDropout1D,
                          Conv2D, MaxPooling2D, UpSampling2D, Lambda)
from tensorflow.keras.models     import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses     import mse, binary_crossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import  mean_squared_error as mse_keras
from tensorflow.keras.backend import argmax as argmax
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import one_hot
from tensorflow.keras.models import Sequential 

from tensorflow.keras.utils import  to_categorical
from tensorflow import random as randomtf

from IPython.display import clear_output
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.lines as mlines
from   matplotlib.lines import Line2D
from   matplotlib.colors import ListedColormap
import matplotlib.ticker as tk

from progressbar import ProgressBar
import seaborn as sns

from chainer_chemistry.dataset.preprocessors import GGNNPreprocessor, construct_atomic_number_array
preprocessor = GGNNPreprocessor()
from rdkit import rdBase
rdBase.DisableLog('rdApp.error')
from rdkit import Chem

import ntpath
from scipy.stats import truncnorm

import sys
sys.path.append("./../utils/")
from general import *

""" fix all the seeds,results are still slighthly different """
randomtf.set_seed(10)
os.environ['PYTHONHASHSEED'] = '10'
np.random.seed(420)
random.seed(123450)
from progressbar import ProgressBar

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.4)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1, gpu_options=gpu_options)
tf.compat.v1.set_random_seed(1234)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)
tf.compat.v1.keras.backend.clear_session()

In [ ]:
# check the font !!!!!!!!!!!!!!!!!!!!!
# switch to Arial
# if not working: delet ~/.catch/matplotlib
plt.rcParams["font.family"] = "Arial"
plt.rcParams['ps.useafm'] = True
plt.rcParams['pdf.fonttype'] = 42
mpl.font_manager.FontManager()
rc('font', weight='bold')
fig, ax = plt.subplots(figsize=(5, 4))
plt.scatter([10, 55], [10, 55])
ax.tick_params(axis='both', length=0, width=1.5, colors='black', grid_alpha=0, labelsize=20)
plt.xlabel('!!!Ariaaaal', fontname='Arial', fontsize=50)

In [ ]:
""" reading and preprocessing data"""
with open('./../data/trainingsets/train_regular_pubqc130K/image_train.pickle', 'rb') as f:
    X_smiles_train0, SMILES_train0, y_train00 = pickle.load(f)
    
with open('./../data/trainingsets/train_regular_pubqc130K/image_test.pickle', 'rb') as f:
    X_smiles_val0, SMILES_val0, y_val00 = pickle.load(f)

with open('./../data/trainingsets/train_regular_pubqc130K/tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)
tokenizer[0] = ' '

with open('./../data/trainingsets/train_regular_pubqc130K/tokenizer_object.pickle', 'rb') as f:
    tokenizer_ = pickle.load(f)


print (X_smiles_train0.shape)
print (X_smiles_val0.shape)

In [ ]:
# reduce the number of samples
# subsampling
idx = np.random.choice(len(y_train00), int(len(y_train00) * 1), replace = False)
X_smiles_train, SMILES_train, y_train0 = (X_smiles_train0[idx], SMILES_train0[idx], y_train00[idx])

idx = np.random.choice(len(y_val00), int(len(y_val00) * 1), replace = False)
X_smiles_val, SMILES_val, y_val0 = (X_smiles_val0[idx], SMILES_val0[idx], y_val00[idx])

print (X_smiles_train.shape)
print (X_smiles_val.shape)

In [ ]:
# Standardized between 0 and 11
gap_min = 0
gap_max = 11
y_val   = (y_val0 -   gap_min) / (gap_max - gap_min)
y_train = (y_train0 - gap_min) / (gap_max - gap_min)
print (max(y_val))
print (max(y_train))

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))

ax.tick_params(axis='both', length=4, width=2, colors='black', grid_alpha=0, labelsize=15)
[i.set_linewidth(2) for i in ax.spines.values()]
plt.xlabel('HOMO-LUMO gap (eV)', fontname='Arial', fontweight = 'bold', fontsize=15)
plt.ylabel('% of samples', fontname='Arial', fontweight = 'bold', fontsize=15)

sns.histplot (y_train0, color='red', label='train', stat='percent', kde=True, bins=50)
sns.histplot (y_val0, color='blue', label='test', stat='percent', kde=True, bins=50, alpha=0.5)
plt.legend()
plt.tight_layout()
plt.savefig('gap_test_train.jpeg', dpi=500)

In [ ]:
## models definition & loading pretrained encoder and decoder 
encoder = load_model('./../data/nns/keep/encoder.h5')
decoder = load_model('./../data/nns/keep/decoder.h5')

class Config:
    
    def __init__(self):
        self.Filters = [256, 128, 64]
        self.genFilters = [128, 128, 128]
        self.upFilters = [(2, 2), (2, 2), (2, 2)]
        
config = Config()

## Generator 
z = Input(shape = (128, ))
y = Input(shape = (1, ))

h = Concatenate(axis = 1)([z, y])
h = Dense(1 * 1 * 128)(h)
R1 = Reshape([1, 1, 128])(h)
R2 = Reshape([1, 1, 128])(h)

for i in range(3):
    R1 = UpSampling2D(size = config.upFilters[i])(R1)
    C1 = Conv2D(filters = config.genFilters[i], 
               kernel_size = 2, 
               strides = 1, 
               padding = 'same')(R1)
    B1 = BatchNormalization()(C1)
    R1 = LeakyReLU(alpha=0.2)(B1)

for i in range(3):
    R2 = UpSampling2D(size = config.upFilters[i])(R2)
    C2 = Conv2D(filters = config.genFilters[i], 
               kernel_size = 2, 
               strides = 1, 
               padding = 'same')(R2)
    B2 = BatchNormalization()(C2)
    R2 = LeakyReLU(alpha=0.2)(B2)
    
R1 = Conv2D(1,
            kernel_size = 3,
            strides = 1,
            padding = 'valid',
            activation = 'tanh')(R1)
R2 = Conv2D(1,
            kernel_size = 3,
            strides = 1,
            padding = 'valid',
            activation = 'tanh')(R2)
generator = Model([z, y], [R1, R2])

## Discriminator 
inp1 = Input(shape = [6, 6, 1])
inp2 = Input(shape = [6, 6, 1])
X1 = Concatenate()([inp1, inp2])
X = Flatten()(X1)
y2 = Concatenate(axis = 1)([X, y])
for i in range(3):
		y2 = Dense(64, activation = 'relu')(y2)
		y2 = LeakyReLU(alpha = 0.2)(y2)
		y2 = Dropout(0.2)(y2)

O_dis = Dense(1, activation = 'sigmoid')(y2)


discriminator = Model([inp1, inp2, y], O_dis)
discriminator.compile(loss = 'binary_crossentropy', 
                      optimizer = Adam(lr = 5e-5, beta_1 = 0.5))
print (discriminator.summary()) 

## Regressor
inp1 = Input(shape = [6, 6, 1])
inp2 = Input(shape = [6, 6, 1])

yr = Concatenate()([inp1, inp2])

tower0 = Conv2D(64, 1, padding = 'same')(yr)
tower1 = Conv2D(64, 1, padding = 'same')(yr)
tower1 = Conv2D(64, 3, padding = 'same')(tower1)
tower2 = Conv2D(32, 1, padding = 'same')(yr)
tower2 = Conv2D(32, 5, padding = 'same')(tower2)
tower3 = MaxPooling2D(3, 1, padding = 'same')(yr)
tower3 = Conv2D(32, 1, padding = 'same')(tower3)
h = Concatenate()([tower0, tower1, tower2, tower3])
h = ReLU()(h)
h = MaxPooling2D(2, 1, padding = 'same')(h)

for i in range(6):
    tower0 = Conv2D(64, 1, padding = 'same')(h)
    tower1 = Conv2D(64, 1, padding = 'same')(h)
    tower1 = Conv2D(64, 3, padding = 'same')(tower1)
    tower2 = Conv2D(32, 1, padding = 'same')(h)
    tower2 = Conv2D(32, 5, padding = 'same')(tower2)
    tower3 = MaxPooling2D(3, 1, padding = 'same')(h)
    tower3 = Conv2D(32, 1, padding = 'same')(tower3)
    h = Concatenate()([tower0, tower1, tower2, tower3])
    h = ReLU()(h)
    if i % 2 == 0 and i != 0:
        h = MaxPooling2D(2, 1, padding = 'same')(h)
h = BatchNormalization()(h)
yr = Flatten()(h)
o = Dropout(0.2)(yr)
o = Dense(128)(o)
o_reg = Dropout(0.2)(o)
o_reg = Dense(1, activation = 'sigmoid')(o_reg)

regressor = Model([inp1, inp2], o_reg)
regressor_top = Model([inp1, inp2], o)

In [ ]:
# Training the Regressor 
# latent vectors from trained Encoder
train_atoms_embedding, train_bonds_embedding, _ = encoder.predict([X_smiles_train], verbose=0)
atoms_embedding, bonds_embedding, _ = encoder.predict([X_smiles_train], verbose=0)
atoms_val, bonds_val, _ = encoder.predict([X_smiles_val], verbose=0)

# No training if the trained model is saved. 
try:
    regressor = load_model('./../data/nns/keep/regressor.h5')
    regressor_top = load_model('./../data/nns/keep/regressor_top.h5')
    regressor.compile(loss = 'mse', optimizer = Adam(5e-7))
    print (".h5 was read")
except:
    print ("no .h5 available")
    regressor.compile(loss = 'mse', optimizer = Adam(5e-7))
    pass
    
history = regressor.fit([atoms_embedding, bonds_embedding], 
              y_train,
              validation_data = ([atoms_val,
                                  bonds_val],
                                 y_val),
              batch_size = 128,
              epochs = 1,
              verbose = 1)
    
# Validating the regressor
# Train
pred_train = regressor.predict([atoms_embedding, bonds_embedding])
pred_train0 = pred_train*(gap_max-gap_min)+gap_min
print('Current R2 on Regressor for train data: {}'.format(r2_score(y_train0, pred_train0.reshape([-1]))))
mse_train = mean_squared_error(y_train0, pred_train0.reshape([-1]))
mae_train = mean_absolute_error(y_train0, pred_train0.reshape([-1]))

#print ('prediction on train: ', pred_train)
#print ('True train: ', y_train)

# Test
pred = regressor.predict([atoms_val, bonds_val])
pred0 = pred*(gap_max-gap_min) + gap_min
print('Current R2 on Regressor for test data: {}'.format(r2_score(y_val0, pred0.reshape([-1]))))
mse_test = mean_squared_error (y_val0, pred0.reshape([-1]))
mae_test = mean_absolute_error (y_val0, pred0.reshape([-1]))

#print ("prediction on test: ", pred )
#print ("True test values: ", y_val)

print ('Train MSE: {}, RMSE: {}, MAE: {}'.format (round(mse_train, 5), 
                                                  round(mse_train**0.5, 5), 
                                                  round(mae_train, 5)))
print ('Test MSE: {}, RMSE: {}, MAE: {}'.format (round(mse_test, 5), 
                                                  round(mse_test**0.5, 5), 
                                                  round(mae_test, 5)))
# Saving the currently trained models
regressor.save('./../data/nns/regressor.h5')
regressor_top.save('./../data/nns/regressor_top.h5')

In [ ]:
print (np.max(pred0))
print (np.max(y_train0))
print (np.max(pred_train0))
print (np.max(y_val0))

In [ ]:
MAE_pred_des = np.round (mean_absolute_error(pred0, y_val0), 4)
print ("MAE_pred_des", MAE_pred_des)
# Fractioned MAE, more normalized
Fractioned_MAE_pred_des = 0
for pred, true in zip(pred0, y_val0):
        Fractioned_MAE_pred_des = Fractioned_MAE_pred_des +  abs(pred-true)/true
Fractioned_MAE_pred_des = Fractioned_MAE_pred_des/(pred0.shape[0])
print ("MAEF_pred_des", Fractioned_MAE_pred_des)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))

plt.plot(list(range(len(history.history['loss']))), 
         history.history['loss'], label='Training loss', linewidth=3,) 
plt.plot(list(range(len(history.history['val_loss']))), 
         history.history['val_loss'], label='Validation loss', linewidth=3,) 

ax.set_xlabel('Epochs', fontsize='20', fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('Loss', fontsize='20', fontname='Arial', fontweight='bold', labelpad=5)

ax.tick_params(direction='out', length=5, width=3, colors='black', grid_alpha=1, labelsize='18')
[i.set_linewidth(2) for i in ax.spines.values()]

plt.title('Regressor loss', fontsize=15, fontname='Arial', fontweight='bold', pad=10)
plt.legend(fontsize=15) 
plt.tight_layout()
plt.savefig("R_loss_0_100.png", dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
plt.rcParams["legend.markerscale"] = 10
plt.scatter (y_train0, pred_train0, color='red', label='Train', alpha=0.6, s=0.05)
plt.scatter ( y_val0, pred0, color='blue', label='Test', alpha=0.6, s=0.05)

ax.set_xlabel('DFT gap (eV)', fontsize='20', fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('Pred. gap (eV)', fontsize='20', fontname='Arial', fontweight='bold', labelpad=5)

ax.tick_params(direction='out', length=5, width=3, colors='black', 
               grid_alpha=1, labelsize='18')

[i.set_linewidth(3) for i in ax.spines.values()]
leg = plt.legend(title='Train: R$^2$={}, MAE={} \nTest: R$^2$={}, MAE={}'.\
           format(round(r2_score(y_train0, pred_train0.reshape([-1])), 2), 
                  round (mae_train, 2),
                  round (r2_score(y_val0, pred0.reshape([-1])), 2), 
                  round (mae_test, 2), ), framealpha=0, title_fontsize=15)
leg._legend_box.align = "left"
plt.xlim(0, 12)
plt.ylim(0, 12)
plt.xticks((1, 3, 5, 7, 9,  11));
plt.yticks((1, 3, 5, 7, 9,  11));
plt.plot([0, 12], [0, 12], '--k', )#color='black')
plt.tight_layout()
plt.savefig('regressor_train_test.jpeg', dpi=300)
plt.rcParams["legend.markerscale"] = 1

In [ ]:
K.clear_session()
## Combined model 
def build_combined(z, y,
                   regressor,
                   regressor_top,
                   discriminator,
                   encoder,
                   decoder):
    discriminator.trainable = False
    regressor_top.trainable = False
    regressor.trainable = False
    encoder.trainable = False
    decoder.trainable = False
    
    atoms_emb, bonds_emb = generator([z, y])
    dec_embedding = Concatenate()([atoms_emb, bonds_emb])
    
    softmax_smiles, _ = decoder([dec_embedding])
    argmax_smiles = argmax (softmax_smiles, axis=2)
    argmax_smiles = Reshape([40])(argmax_smiles)
    smiles = one_hot(argmax_smiles, depth=27)
    smiles = Reshape([40, 27, 1])(smiles)
    latent_encoder_atom, latent_encoder_bond, _ = encoder ([smiles])
    
    y_pred = regressor([latent_encoder_atom, latent_encoder_bond])
    valid = discriminator([atoms_emb, bonds_emb, y])
    #print ('valid from comb', valid)

    combined = Model([z, y], [valid, y_pred])

    combined.compile(loss = ['binary_crossentropy',
                             'mse'], 
                     loss_weights = [0.01, 25.0], 
                     optimizer = Adam(5e-6, beta_1 = 0.5))
    
    return combined

combined = build_combined(z, y,
                          regressor,
                          regressor_top,
                          discriminator,
                          encoder,
                          decoder)

In [ ]:
""" Training RCGAN """
# loading pretrained models
regressor = load_model    ('./../data/nns/regressor.h5')
regressor_top = load_model('./../data/nns/regressor_top.h5')
#generator = load_model    ('./../data/nns/keep/generator.h5')
#discriminator = load_model ('./../data/nns/keep/discriminator.h5')
#combined = load_model ('./../data/nns/combined.h5')

regressor_top.trainable = False
regressor.trainable = False

p = multiprocessing.Process()

# SMILES related information
max_gen_atoms = 9
bond_max = 9
MAX_NB_WORDS = 27
MAX_SEQUENCE_LENGTH = 40

epochs = 150
batch_size = 256
batches = y_train0.shape[0] // batch_size
threshold = 0.2 # defining accurate samples
reinforce_n = 50 # 5*reinforce_n = fake sampling
reinforce_sample = 1000 # how many samples generated for Reinforcement

# variable for storing generated data
G_Losses = []
D_Losses = []
R_Losses = []
D_Losses_real = []
D_Losses_fake = []

for e in range(epochs):
    start = time.time()
    D_loss = []
    G_loss = []
    R_loss = []
    D_loss_real = []
    D_loss_fake = []
    
    for b in range(batches):
        
        regressor_top.trainable = False
        regressor.trainable = False

        idx = np.arange(b * batch_size, (b + 1) * batch_size)
        # shuffle samples 
        idx = np.random.choice(idx, batch_size, replace = False)
        
        x_smiles_train = X_smiles_train[idx] 
        batch_y = y_train[idx]
        
        batch_z = np.random.normal(0, 1, size = (batch_size, 128))
        
        atoms_embedding, bonds_embedding, _ = encoder.predict([x_smiles_train], verbose=0)
        dec_embedding = np.concatenate([atoms_embedding, bonds_embedding], axis = -1)
        
        gen_atoms_embedding, gen_bonds_embedding = generator.predict([batch_z, batch_y], verbose=0)
        
        gen_dec_embedding = np.concatenate([gen_atoms_embedding, gen_bonds_embedding], axis = -1)
        softmax_smiles = decoder.predict(gen_dec_embedding, verbose=0)[0]
        
        argmax_smiles = np.argmax(softmax_smiles, axis = 2)
        smiles = to_categorical(argmax_smiles, num_classes=27)
        SHAPE = list(smiles.shape) + [1]
        smiles = smiles.reshape(SHAPE)
        latent_encoder_atom, latent_encoder_bond, _ = encoder.predict([smiles], verbose=0)
        gen_pred = regressor.predict([latent_encoder_atom, latent_encoder_bond], verbose=0).reshape([-1])
        
        regressor.trainable = True
        r_loss = regressor.train_on_batch([atoms_embedding, bonds_embedding], batch_y)
        R_loss.append(r_loss)
        regressor.trainable = False

        discriminator.trainable = True

        d = 3
        for _ in range(d):
            d_loss_real = discriminator.train_on_batch([atoms_embedding, bonds_embedding, batch_y],
                                                       [0.9 * np.ones((batch_size, 1))])
            d_loss_fake = discriminator.train_on_batch([gen_atoms_embedding, gen_bonds_embedding, batch_y],
                                                       [np.zeros((batch_size, 1))]) 

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        D_loss.append(d_loss)
        D_loss_real.append (d_loss_real)
        D_loss_fake.append (d_loss_fake)
        discriminator.trainable = False
        
        regressor_top.trainable = False
        regressor.trainable = False

        g_loss = combined.train_on_batch([batch_z, batch_y], [0.9 * np.ones((batch_size, 1)), batch_y])
        G_loss.append(g_loss[0])
    
    D_Losses.append(np.mean(D_loss))
    D_Losses_real.append(np.mean(D_loss_real))
    D_Losses_fake.append(np.mean(D_loss_fake))
    G_Losses.append(np.mean(G_loss))
    R_Losses.append(np.mean(R_loss))
    
    print('====')
    print('Current epoch: {}/{}'.format((e + 1), epochs))
    print ('D Loss Real: {}'.format(np.mean(D_loss_real)))
    print ('D Loss Fake: {}'.format(np.mean(D_loss_fake)))
    print('D Loss: {}'.format(np.mean(D_loss)))
    print('G Loss: {}'.format(np.mean(G_loss)))
    print('R Loss: {}'.format(np.mean(R_loss)))
    print('====')
    print()

    
    # Reinforcement
    gen_error = []
    gen_smiles = []
    gen_valid_smiles = []
    gen_X_atoms = []
    gen_X_bonds = []
    predcv_AE_latent = []
    embeddings = []
    sample_ys = []
    valid_smiles_index = []
    for _ in range(reinforce_sample):
        sample_y = np.random.uniform(gap_min, gap_max, size = [1, ])
        sample_y = np.round(sample_y, 4)
        sample_y = (sample_y - gap_min) / (gap_max - gap_min)
        sample_ys.append(sample_y)

        sample_z = np.random.normal(0, 1, size = (1, 128))

        sample_atoms_embedding, sample_bonds_embedding = generator.predict([sample_z, sample_y], verbose=0)
        embeddings.append((sample_atoms_embedding, sample_bonds_embedding))
        
        dec_embedding = np.concatenate([sample_atoms_embedding, sample_bonds_embedding], axis = -1)
        softmax_smiles = decoder.predict(dec_embedding, verbose=0)[0]
        argmax_smiles = np.argmax(softmax_smiles, axis = 2).reshape([-1])
        smiles = to_categorical(argmax_smiles, num_classes=27)
        SHAPE = [1] + list(smiles.shape) + [1]
        smiles = smiles.reshape(SHAPE)
        c_smiles = ''
        for s in argmax_smiles:
            c_smiles += tokenizer[s]
        c_smiles = c_smiles.rstrip()
        
        gen_smiles.append(c_smiles)
        latent_encoder_atom, latent_encoder_bond, _ = encoder.predict([smiles], verbose=0)
        reg_pred = regressor.predict([latent_encoder_atom, latent_encoder_bond], verbose=0)
        pred, desire = reg_pred[0][0], sample_y[0]

        gen_error.append(round (np.abs((pred - desire) / desire), 5))

        
    gen_error = np.asarray(gen_error).reshape(-1, 1)
    # two validity defined: 
    ## without sanitizing: valid0, Sanitized: valid
    valid = 0
    valid0 = 0
    idx_ = []
    idx0_ = []
    preds_can = []
    for iter_, smiles in enumerate(gen_smiles):
        if ' ' in smiles[:-1]:
            continue
        m  = Chem.MolFromSmiles(smiles[:-1], sanitize=True)
        m0 = Chem.MolFromSmiles(smiles[:-1], sanitize=False)
        if m0 is not None:
            valid0 += 1
            idx0_.append(iter_)
        if m is not None:
            valid += 1
            idx_.append(iter_)
            gen_smiles [iter_] = Chem.MolToSmiles(m, canonical=True)
            smiles_can = Chem.MolToSmiles(m, canonical=True)
            smiles_can_dot = smiles_can + '.'
            X_smiles0 = tokenizer_.texts_to_sequences([smiles_can_dot])
            X_smiles1 = pad_sequences(X_smiles0, maxlen = 40, padding = 'post')
            X_smiles2 = to_categorical(X_smiles1, num_classes=27)
            latent_encoder_atom, latent_encoder_bond, _ = encoder.predict(X_smiles2, verbose=0)
            pred_can = regressor.predict([latent_encoder_atom, latent_encoder_bond], verbose=0).reshape([-1])
            #pred_can = pred_can*11
            preds_can.append(pred_can[0])
            print (Chem.MolToSmiles(m, canonical=True))
            print ("gap_des", sample_ys[iter_])
            print ("error", gen_error[iter_])

    idx_ = np.asarray(idx_)

    idx0_ = np.asarray(idx0_)
    preds_can = np.array (preds_can).reshape(-1, 1)
    #print('gen error before correction:', gen_error [idx_])
    sample_ys_ = np.array (sample_ys)
    gen_error [idx_] = [(abs(i-j)/j) for i,j in zip(preds_can, sample_ys_[idx_])] 
    #print('gen error after correction:', gen_error [idx_])
    validity = [gen_smiles[jj] for jj in idx0_ ]
    validity = pd.DataFrame(validity)
    validity = validity.drop_duplicates()

    validity_sanitize = [gen_smiles[jj] for jj in idx_ ]
    validity_sanitize = pd.DataFrame(validity_sanitize)
    validity_sanitize = validity_sanitize.drop_duplicates()

    if (e + 1) % 100 == 0:
        reinforce_n += 10

    # invalid smiles:
    fake_indices1 = np.setdiff1d(np.arange(reinforce_sample), np.asarray(idx_))
    fake_indices2 = np.intersect1d(np.where(gen_error > threshold)[0], idx_)
    fake_indices = np.concatenate ((fake_indices1, fake_indices2))
    fake_indices = np.random.choice(fake_indices, reinforce_n * 5, replace = False)

    real_indices_ = np.intersect1d(np.where(gen_error <= threshold)[0], idx_)
    sample_size =  len(real_indices_)
    real_indices = np.random.choice(real_indices_, sample_size, replace = False)
    
    # Activating Reinforcement 
    if e >= 5:
        discriminator.trainable = True
        regressor_top.trainable = False
        regressor.trainable = False
        for real_index in real_indices:
            _ = discriminator.train_on_batch([embeddings[real_index][0], 
                                              embeddings[real_index][1], 
                                              sample_ys[real_index]],
                                             [1 * np.ones((1, 1))])

        for fake_index in fake_indices:
            _ = discriminator.train_on_batch([embeddings[fake_index][0], 
                                              embeddings[fake_index][1] , 
                                              sample_ys[fake_index]],
                                             [np.zeros((1, 1))])
        discriminator.trainable = False

    # ==== #
    try:
        print('Currently valid SMILES (No chemical_beauty and sanitize off): {}'.format(valid0))
        print('Currently valid SMILES Unique (No chemical_beauty and sanitize off): {}'.format(len(validity)))
        print('Currently valid SMILES Sanitized: {}'.format(valid))
        print('Currently valid Unique SMILES Sanitized: {}'.format(len(validity_sanitize)))
        print('Currently satisfying SMILES: {}'.format(len(real_indices_)))
        print('Currently unique satisfying generation: {}'.format(len(np.unique(np.array(gen_smiles)[real_indices_]))))
        print('====')
        print()
    except:
        pass
    
    if (e + 1) % 5 == 0:
        plt.close()
        fig, ax = plt.subplots(figsize = (12, 10))
        ax.tick_params(axis='both', which='major', labelsize=30)
        plt.plot(G_Losses, color='blue')
        plt.plot(D_Losses, color='red')
        plt.xlabel('epochs', fontsize=35)
        plt.ylabel('loss', fontsize=35)
        mpl.rcParams['axes.linewidth'] = 2.5
        plt.legend(['G Loss', 'D Loss'], fontsize=30)
        plt.savefig("G_D_losses{}.png".format (e+1))
    

    n_unique = len(np.unique(np.array(gen_smiles)[real_indices_]))
    n_valid = valid

    end = time.time()
    print ("time for current epoch: ", (end - start))
    tf.compat.v1.keras.backend.clear_session()

with open('GAN_loss.pickle', 'wb') as f:
    pickle.dump((G_Losses, D_Losses, R_Losses), f)

# Saving the currently trained models
#regressor.save('regressor.h5')
#regressor_top.save('regressor_top.h5')
generator.save('./../data/nns/generator.h5')
discriminator.save('./../data/nns/discriminator.h5')
combined.save('./../data/nns/combined.h5')

p.start()
pred_can
p.join()
print ('Done')
tf.compat.v1.keras.backend.clear_session()

In [ ]:
encoder = load_model('./../data/nns/keep/encoder.h5')
decoder = load_model('./../data/nns/keep/decoder.h5')
model = load_model('./../data/nns/keep/ae_model.h5')

regressor = load_model    ('./../data/nns/keep/regressor.h5')
regressor_top = load_model('./../data/nns/keep/regressor_top.h5')
generator = load_model    ('./../data/nns/keep/generator.h5')
discriminator= load_model ('./../data/nns/keep/discriminator.h5')

pbar = ProgressBar()
max = 0.3

randS = []
rsquaredS = []
MAE_S = []
less20RE_perS = []
output_lenS = []
mean_RE_S = []
for rand in pbar(range(0, 1)):  
    N = 50
    n_sample = 75
    gen_error = []
    gen_smiles = []
    sample_ys = []
    preds = []
  
    predss_can = []
    gen_atoms_embedding = []
    gen_bonds_embedding = []

    regressor_top.trainable = False
    regressor.trainable = False
    generator.trainable = False
    discriminator.trainable = False

    np.random.seed(rand)

    pbar = ProgressBar()
    sample_y = np.random.uniform(0, 10.8, size=[n_sample, ])
    for hc in (pbar(range(hc))):
        try:
            # get it back to original of s_min to s_max
            #sample_y = np.random.uniform(0, 15, size=[1,])
            sample_y = hc
            #print (sample_y)
            sample_y = np.round(sample_y, 4)
            sample_y = sample_y * np.ones([N, ])
            sample_y_ = (sample_y - gap_min) / (gap_max - gap_min)
            sample_z = np.random.normal(0, 1, size = (N, 128))

            regressor_top.trainable = False
            regressor.trainable = False
            encoder.trainable = False
            decoder.trainable = False

            sample_atoms_embedding, sample_bonds_embedding = generator.predict([sample_z, sample_y_], verbose=0)
            dec_embedding = np.concatenate([sample_atoms_embedding, sample_bonds_embedding], axis = -1)

            softmax_smiles = decoder.predict(dec_embedding, verbose=0)[0]
            argmax_smiles = np.argmax(softmax_smiles, axis = 2)
            #print (argmax_smiles)

            #print ('shape argmax_smiles', argmax_smiles.shape)
            smiles = to_categorical(argmax_smiles, num_classes=27)
            
            SHAPE = list(smiles.shape) + [1] 
            
            #print ('shape line 767', SHAPE) 
            smiles = smiles.reshape(SHAPE)

            latent_encoder_atom, latent_encoder_bond, _ = encoder.predict([smiles], verbose=0)
            pred = regressor.predict([latent_encoder_atom, latent_encoder_bond], verbose=0).reshape([-1])
            pred = pred * (gap_max - gap_min) + gap_min

            gen_errors = np.abs((pred - sample_y) / sample_y).reshape([-1])


            smiles = decoder.predict(dec_embedding, verbose=0)[0]
            #print(smiles)
            smiles = np.argmax(smiles, axis = 2).reshape(smiles.shape[0], 40)
            

            generated_smiles = []
            
            for S in smiles:
                c_smiles = ''
                for s in S:
                    c_smiles += tokenizer[s]
                c_smiles = c_smiles.rstrip()
                #print (c_smiles)
                generated_smiles.append(c_smiles)
            generated_smiles = np.array(generated_smiles)
            #generated_smiles = generated_smiles [accurate]
            all_gen_smiles = []
            idx = []
            preds_can = []
            for i, smiles in enumerate(generated_smiles):
                all_gen_smiles.append(smiles[:-1])

                if ' ' in smiles[:-1]:
                    continue
                #m = Chem.MolFromSmiles(smiles[:-1], sanitize=False)
                m = Chem.MolFromSmiles(smiles[:-1], sanitize=True)
                if m is not None:
                    
                    smiles_can = Chem.MolToSmiles(m, canonical=True)
                    smiles_can_dot = smiles_can + '.'
                    X_smiles0 = tokenizer_.texts_to_sequences([smiles_can_dot])
                    X_smiles1 = pad_sequences(X_smiles0, maxlen=40, padding='post')
                    X_smiles2 = to_categorical(X_smiles1, num_classes=27)
                    latent_encoder_atom, latent_encoder_bond, _ = encoder.predict(X_smiles2, verbose=0)
                    pred_can = regressor.predict([latent_encoder_atom, latent_encoder_bond], verbose=0).reshape([-1])
                    pred_can = pred_can*11
                    if pred_can >= 9.5:
                        preds_can.append(pred_can[0])
                        idx.append(i)

            idx = np.array(idx)
            all_gen_smiles = np.array(all_gen_smiles)
            gen_smiles.extend(list(all_gen_smiles[idx]))
            gen_error.extend(list(gen_errors[idx]))
            sample_ys.extend(list(sample_y[idx]))
            gen_atoms_embedding.extend(sample_atoms_embedding[idx])
            gen_bonds_embedding.extend(sample_bonds_embedding[idx])
            preds.extend(list(pred[idx]))
            predss_can.extend(list(preds_can))
            tf.compat.v1.keras.backend.clear_session()
        except:
            tf.compat.v1.keras.backend.clear_session()
            pass    


    output = {}

    for i, s in enumerate (gen_smiles):
        ss = Chem.MolToSmiles(Chem.MolFromSmiles(s, sanitize=True), canonical=True)
        gen_smiles[i] = ss

    tf.compat.v1.keras.backend.clear_session()
    output['SMILES'] = gen_smiles
    #output['des_gap'] = sample_ys
    # More accurate for regressor to predict gap from canonical SMILES
    output['pred_gap'] = predss_can
    #output['Err_pred_des'] = gen_error
    #output['Err_pred_des'] = [abs(i- j)/i for i, j in zip(output['des_gap'], output['pred_gap'])]
    output = pd.DataFrame(output)
    output.reset_index(drop = True, inplace = True)

    output.to_csv ('./../experiments/regular/outliers{}.csv'.format(random.randint(100, 10000)), index=False)

    ## Statistics  (# pred=True value, Des=prediction)
    # total # of samples
    N = len(predss_can)
    print ('random seed', rand)    
    tf.compat.v1.keras.backend.clear_session()